# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [25]:
# Write your code below.

from dotenv import load_dotenv
import os

# Load the .env file (make sure the path is correct)
load_dotenv('C:/Tina Lin/Training/DSI/Week10_Production/assignments/.env') 

my_variable = os.getenv("TICKERS") 
print(my_variable)


../../05_src/data/tickers/sp500_wiki.csv


In [26]:
import sys
sys.path.append("../../05_src")

In [13]:
import logging
#from logger import get_logger
_logs = logging.getLogger(__name__)
_logs.info("Hello world!")

In [27]:
import dask.dataframe as dd


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [15]:
import os
import glob

# Write your code below.

print(type(glob)) 


<class 'module'>


In [31]:
# Get the PRICE_DATA environment variable
price_data_dir = os.getenv("PRICE_DATA")

# Print the PRICE_DATA directory to verify it is loaded correctly
print("PRICE_DATA directory:", price_data_dir)

# Convert relative path to absolute path
absolute_price_data_dir = os.path.abspath(price_data_dir)
print("Absolute PRICE_DATA directory:", absolute_price_data_dir)

# Use glob to find all parquet files in the PRICE_DATA directory (absolute path)
parquet_files = glob.glob(os.path.join(absolute_price_data_dir, "*.parquet"))

# Use glob to find all parquet files in the PRICE_DATA directory
#parquet_files = glob.glob(os.path.join(price_data_dir, "*.parquet"))

# Print the list of parquet files found
print("Parquet files found:", parquet_files)

PRICE_DATA directory: ../../05_src/data/prices/
Absolute PRICE_DATA directory: c:\Tina Lin\Training\DSI\05_src\data\prices
Parquet files found: ['c:\\Tina Lin\\Training\\DSI\\05_src\\data\\prices\\sample_tickers.parquet']


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [34]:
import pandas as pd

# Load the CSV file
csv_file_path = "c:/Tina Lin/Training/DSI/05_src/data/prices/sample_tickers.csv"
df = pd.read_csv(csv_file_path)

# Convert and save as a Parquet file
parquet_file_path = "c:/Tina Lin/Training/DSI/05_src/data/prices/sample_tickers.parquet"
df.to_parquet(parquet_file_path, index=False)

print("CSV file converted to Parquet successfully.")

CSV file converted to Parquet successfully.


In [35]:
import pandas as pd

# Try to load as a Parquet file
try:
    df = pd.read_parquet("c:/Tina Lin/Training/DSI/05_src/data/prices/sample_tickers.parquet")
    print(df.head())
except Exception as e:
    print("Error loading as Parquet:", e)

  ticker                 Security             GICS Sector  \
0   AAPL               Apple Inc.  Information Technology   
1  GOOGL  Alphabet Inc. (Class A)  Communication Services   
2   GOOG  Alphabet Inc. (Class C)  Communication Services   
3   NVDA                   Nvidia  Information Technology   
4   MSFT          Microsoft Corp.  Information Technology   

                            GICS Sub-Industry      Headquarters Location  \
0  Technology Hardware, Storage & Peripherals      Cupertino, California   
1                Interactive Media & Services  Mountain View, California   
2                Interactive Media & Services  Mountain View, California   
3                              Semiconductors    Santa Clara, California   
4                        Application Software        Redmond, Washington   

   Date added      CIK Founded  
0       30285   320193    1977  
1       41732  1652044    1998  
2       38810  1652044    1998  
3       37225  1045810    1993  
4       301

In [38]:
# Write your code below.

import dask.dataframe as dd

# Load the data 
dd_data = dd.read_parquet("c:/Tina Lin/Training/DSI/05_src/data/prices/sample_tickers.parquet")

# Print the columns to check if 'Close' exists
print("Columns in DataFrame:", dd_data.columns)

# Check the first few rows to see the data
print("First few rows of DataFrame:")
print(dd_data.head())

# Ensure the 'Close' column exists before proceeding
if 'Close' in dd_data.columns:
    # Add lags for variables Close and Adj_Close (if Adj_Close exists)
    dd_data['Close_lag'] = dd_data['Close'].shift(1)
    if 'Adj_Close' in dd_data.columns:
        dd_data['Adj_Close_lag'] = dd_data['Adj_Close'].shift(1)
        # Calculate returns
        dd_data['returns'] = (dd_data['Adj_Close'] / dd_data['Adj_Close_lag']) - 1
    else:
        print("'Adj_Close' column not found.")
else:
    print("'Close' column not found.")

# Calculate the range (High - Low)
if 'High' in dd_data.columns and 'Low' in dd_data.columns:
    dd_data['hi_lo_range'] = dd_data['High'] - dd_data['Low']
else:
    print("'High' or 'Low' column not found.")

# Assign the resulting DataFrame to dd_feat
dd_feat = dd_data


Columns in DataFrame: Index(['ticker', 'Security', 'GICS Sector', 'GICS Sub-Industry',
       'Headquarters Location', 'Date added', 'CIK', 'Founded'],
      dtype='object')
First few rows of DataFrame:
  ticker                 Security             GICS Sector  \
0   AAPL               Apple Inc.  Information Technology   
1  GOOGL  Alphabet Inc. (Class A)  Communication Services   
2   GOOG  Alphabet Inc. (Class C)  Communication Services   
3   NVDA                   Nvidia  Information Technology   
4   MSFT          Microsoft Corp.  Information Technology   

                            GICS Sub-Industry      Headquarters Location  \
0  Technology Hardware, Storage & Peripherals      Cupertino, California   
1                Interactive Media & Services  Mountain View, California   
2                Interactive Media & Services  Mountain View, California   
3                              Semiconductors    Santa Clara, California   
4                        Application Software     

+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [40]:
# Write your code below.

import dask.dataframe as dd
import pandas as pd

# Load the Dask DataFrame (ensure you have already done this)
dd_data = dd.read_parquet("c:/Tina Lin/Training/DSI/05_src/data/prices/sample_tickers.parquet")

# Check for relevant columns
print("Columns in DataFrame:", dd_data.columns)

# Adding lag columns for Close and Adj_Close
if 'Close' in dd_data.columns:
    dd_data['Close_lag'] = dd_data['Close'].shift(1)
else:
    print("'Close' column not found.")

if 'Adj_Close' in dd_data.columns:
    dd_data['Adj_Close_lag'] = dd_data['Adj_Close'].shift(1)
else:
    print("'Adj_Close' column not found.")

# Calculate returns if the lagged column exists
if 'Adj_Close_lag' in dd_data.columns:
    dd_data['returns'] = (dd_data['Adj_Close'] / dd_data['Adj_Close_lag']) - 1
else:
    print("'Adj_Close_lag' column not found.")

# Now convert to a Pandas DataFrame if 'returns' was successfully created
if 'returns' in dd_data.columns:
    df = dd_data.compute()  # Convert to Pandas DataFrame
    print("Returns calculated successfully.")

    # Add a rolling average return calculation with a window of 10 days
    df['rolling_avg_return'] = df['returns'].rolling(window=10).mean()

    # Display the updated DataFrame
    print(df.head())
else:
    print("Returns calculation failed due to missing columns.")

Columns in DataFrame: Index(['ticker', 'Security', 'GICS Sector', 'GICS Sub-Industry',
       'Headquarters Location', 'Date added', 'CIK', 'Founded'],
      dtype='object')
'Close' column not found.
'Adj_Close' column not found.
'Adj_Close_lag' column not found.
Returns calculation failed due to missing columns.


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

+ Was it necessary to convert to pandas to calculate the moving average return?

Answer: It is not necessary to convert a Dask DataFrame to a Pandas DataFrame to calculate the moving average return. Dask supports rolling computations directly on Dask DataFrames.

+ Would it have been better to do it in Dask? Why?

Answer: Yes. Using Dask for large datasets is better due to its efficiency, parallelism, and ability to handle data that exceeds memory limits. If the dataset is manageable in size, we might still choose Pandas for simplicity in some cases, but for scalability and performance, Dask is the preferred option for larger datasets.



## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.